In [91]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [92]:
train = pd.read_csv("../processed/train_last_6.csv")
test = pd.read_csv("../processed/test_last_6.csv")

In [93]:
train_x = train.iloc[:, :12]
train_y = train.iloc[:, 12]

In [94]:
test_x = test.iloc[:, :12]
test_y = test.iloc[:, 12]

In [95]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(train_y)
label_encoded_train_y = label_encoder.transform(train_y)
label_encoded_test_y = label_encoder.transform(test_y)

In [96]:
xg_train = xgb.DMatrix(train_x, label=label_encoded_train_y, 
                       feature_names=['atga_last_6', 'atgf_last_6', 'atsa_last_6', 'atsf_last_6', 
                                      'atsta_last_6', 'atstf_last_6', 
                                      'htga_last_6', 'htgf_last_6', 'htsa_last_6', 'htsf_last_6', 
                                      'htsta_last_6', 'htstf_last_6'])
xg_test = xgb.DMatrix(test_x, label=label_encoded_test_y,
                     feature_names=['atga_last_6', 'atgf_last_6', 'atsa_last_6', 'atsf_last_6', 
                                      'atsta_last_6', 'atstf_last_6', 
                                      'htga_last_6', 'htgf_last_6', 'htsa_last_6', 'htsf_last_6', 
                                      'htsta_last_6', 'htstf_last_6'])

In [295]:
param = {'max_depth': 6, 'objective': 'multi:softmax', 'num_class': 3, 'eval_metric': ['mlogloss', 'merror'],
         'eta': 0.05,
         'booster': 'dart'}
param['nthread'] = 8

In [296]:
evallist = [(xg_train, 'train'), (xg_test, 'test')]

In [297]:
num_round = 3
bst = xgb.train(param, xg_train, num_round, evallist)
preds = bst.predict(xg_test)


[20:01:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[20:01:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[20:01:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[20:01:36] src/gbm/gbtree.cc:494: drop 0 trees, weight = 1
[0]	train-mlogloss:1.08697	train-merror:0.431935	test-mlogloss:1.09045	test-merror:0.508594
[20:01:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[20:01:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=6
[20:01:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[20:01:36] src/gbm/gbtree.cc:494: drop 0 trees, weight = 1
[1]	train-mlogloss:1.07595	train-merror:0.427899	test-mlogloss:1.08287	test-merror:0.492969
[20:01:36] src/tree/up

In [298]:
preds.sum()

1872.0

In [299]:
from sklearn.metrics import confusion_matrix

In [300]:
label_encoded_test_y

array([2, 2, 1, ..., 0, 0, 2])

In [301]:
confusion_matrix(label_encoded_test_y, preds)

array([[152,   4, 207],
       [ 89,   8, 222],
       [ 91,  12, 495]])

In [302]:
from sklearn.metrics import accuracy_score

In [303]:
accuracy_score(label_encoded_test_y, preds)

0.51171875

In [304]:
train_preds = bst.predict(xg_train)

In [305]:
accuracy_score(label_encoded_train_y, train_preds)

0.5723256335501233

In [306]:
preds.min()

0.0

In [307]:
preds.max()

2.0

In [281]:
results = pd.DataFrame([label_encoder.inverse_transform([int(x) for x in preds])]).transpose()

TypeError: only size-1 arrays can be converted to Python scalars

In [269]:
probs = bst.predict

,0
0,H
1,H
2,A
3,H
4,A
5,H
6,H
7,H
8,H
9,H


In [ ]:
res